In [2]:
import open3d as o3d  
import numpy as np  
import laspy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import cKDTree
from sklearn import linear_model
from sklearn.cluster import KMeans

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
las = laspy.read('media/KU_003.las')
list(las.point_format.dimension_names)
set(list(las.classification))

print('Points from data:', len(las.points))
print("No.of classification", np.unique(las.classification))


# To create 3D point cloud data, we can stack together with the X, Y, and Z dimensions, using Numpy like this.
point_data = np.stack([las.X, las.Y, las.Z], axis=0).transpose()
print(point_data)


# OPen3D Point Cloud Visualization Colored
geom = o3d.geometry.PointCloud()
geom.points = o3d.utility.Vector3dVector(point_data)
o3d.visualization.draw_geometries([geom])


# Create Open3D point cloud Binary
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(np.vstack((las.x, las.y, las.z)).transpose())
point_cloud.colors = o3d.utility.Vector3dVector(np.vstack((las.intensity, las.intensity, las.intensity)).transpose() / np.max(las.intensity))
#o3d.visualization.draw_geometries([point_cloud])

# # Set point size to make points smaller
# point_size = 0.03  # Adjust this value to make points smaller or larger

# # Create Open3D visualizer
# vis = o3d.visualization.Visualizer()
# vis.create_window()

# # Add point cloud to the visualizer
# vis.add_geometry(point_cloud)

# # Set point size in the render options
# render_opt = vis.get_render_option()
# render_opt.point_size = point_size

# # Render and display the point cloud
# vis.run()
# vis.destroy_window()

Points from data: 3814230
No.of classification [1 2]
[[ 57173998 887874563      3697]
 [ 57173984 887874550      3725]
 [ 57173870 887874608      3835]
 ...
 [ 57159475 887864292      4163]
 [ 57159690 887864035      4189]
 [ 57159803 887864297      4205]]


In [21]:
xyz = np.stack((las.x, las.y, las.z), axis=-1)  # Formatting of numpy array

if xyz.size > 10:
    XY = xyz[:, :2]  # In 3 range column array extracted 2 columns
    Z = xyz[:, 2]  # All but only index[2] extracted

    # Use RANSAC to fit a plane to the point cloud
    ransac = linear_model.RANSACRegressor(residual_threshold=6)
    ransac.fit(XY, Z)

    # Identify inliers (ground points) and outliers
    inlier_mask = ransac.inlier_mask_ #identify inliers
    ground_points = xyz[inlier_mask] #---------------inliers ground truth
    outlier_mask = np.logical_not(inlier_mask) #in logical not inliers are all outliers.
    outliers = xyz[outlier_mask] #--------------outliers container  

    # Extract plane parameters
    plane_coefficients = ransac.estimator_.coef_ #coefficient
    plane_intercept = ransac.estimator_.intercept_ #value of d
    
    # Create a plane model filled with zeros not the same from original cloud data point but the same shape with 
    #lidar pointcloud
    fitted_plane = np.zeros_like(xyz)
    fitted_plane[:, :2] = XY
    fitted_plane[:, 2] = ransac.predict(XY)

    # Visualize ground points and fitted plane using Open3D
    if len(fitted_plane) > 0:
        fitted_plane_pc = o3d.geometry.PointCloud()
        fitted_plane_pc.points = o3d.utility.Vector3dVector(fitted_plane)
    
        # Ensure that the indices are integers (or boolean) for proper indexing
        intensity_indices = np.asarray(inlier_mask).nonzero()[0]

        # Use intensity_indices for indexing las.intensity
        fitted_plane_pc.colors = o3d.utility.Vector3dVector(np.vstack((las.intensity[intensity_indices], las.intensity[intensity_indices], las.intensity[intensity_indices])).transpose() / np.max(las.intensity))
        o3d.visualization.draw_geometries([fitted_plane_pc])

    # Print the plane equation
    print(f"Fitted plane equation: Z = {plane_coefficients[0]} * X + {plane_coefficients[1]} * Y + {plane_intercept}")

        
    # Visualize ground points and outliers using Open3D with intensity-based colors
    if len(ground_points) > 0 and len(outliers) > 0:
        ground_pc = o3d.geometry.PointCloud()
        ground_pc.points = o3d.utility.Vector3dVector(ground_points)
        #ground_pc.colors = o3d.utility.Vector3dVector(np.vstack((las.intensity[inlier_mask], las.intensity[inlier_mask], las.intensity[inlier_mask])).transpose() / np.max(las.intensity))
        o3d.visualization.draw_geometries([ground_pc])
        
        outliers_pc = o3d.geometry.PointCloud()
        outliers_pc.points = o3d.utility.Vector3dVector(outliers)
        outliers_pc.colors = o3d.utility.Vector3dVector(np.vstack((las.intensity[outlier_mask], las.intensity[outlier_mask], las.intensity[outlier_mask])).transpose() / np.max(las.intensity))
        o3d.visualization.draw_geometries([outliers_pc])

        # Print the number of ground points, outliers, and total points
        num_ground_points = len(ground_points)
        num_outliers = len(outliers)
        total_points = len(xyz)
        print(f"Number of ground points: {num_ground_points}")
        print(f"Number of outliers: {num_outliers}")
        print(f"Total number of points in LAS file: {total_points}")
else:
    print("Insufficient points for processing.")


Fitted plane equation: Z = 0.001462977698064171 * X + -0.009544437737279364 * Y + 83943.62366060691
Number of ground points: 3498011
Number of outliers: 316219
Total number of points in LAS file: 3814230


In [6]:
# Load outlier coordinates from the CSV file
outliers_csv_path = 'outliers.csv'
outliers = np.loadtxt(outliers_csv_path, delimiter=',')

# Perform k-means clustering
kmeans = KMeans(n_clusters=150)  # You can adjust the number of clusters (n_clusters) as needed
kmeans.fit(outliers)
labels = kmeans.labels_

# Add cluster labels to the outliers array
outliers_with_labels = np.column_stack((outliers, labels))

# Filter out outliers (points not assigned to any cluster)
clustered_points = outliers_with_labels[outliers_with_labels[:, 3] != -1, :3]

# Create an Open3D point cloud from the clustered points with cluster labels
clustered_pc = o3d.geometry.PointCloud()
clustered_pc.points = o3d.utility.Vector3dVector(clustered_points)

# Assign unique colors to each cluster
unique_colors = np.random.rand(kmeans.n_clusters, 3)
cluster_colors = unique_colors[labels[labels != -1]]

clustered_pc.colors = o3d.utility.Vector3dVector(cluster_colors)

# Visualize the clustered points with cluster-specific colors using Open3D
o3d.visualization.draw_geometries([clustered_pc])

# Create bounding boxes for each cluster
bounding_boxes = []
for cluster_label in range(kmeans.n_clusters):
    cluster_points = clustered_points[labels[labels != -1] == cluster_label]
    
    # Calculate bounding box parameters
    min_bound = np.min(cluster_points, axis=0)
    max_bound = np.max(cluster_points, axis=0)
    
    # Create bounding box
    bounding_box = o3d.geometry.OrientedBoundingBox.create_from_points(o3d.utility.Vector3dVector(cluster_points))
    bounding_box.color = unique_colors[cluster_label].tolist()  # Set color to the cluster color
    
    bounding_boxes.append(bounding_box)

# Visualize all bounding boxes together
o3d.visualization.draw_geometries([clustered_pc] + bounding_boxes)
#it is useful for drone autonomous navigation

/home/x/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
